# *tridesclous* example with olfactory bulb dataset



In [2]:
%matplotlib inline

import time
import numpy as np
import matplotlib.pyplot as plt
import tridesclous as tdc

from tridesclous import DataIO, CatalogueConstructor, Peeler

# DataIO = define datasource and working dir

trideclous provide some datasets than can be downloaded.

Note this dataset contains 3 trials in 3 different files. (the original contains more!)

Each file is considers as a *segment*. *tridesclous* automatically deal with it.

Theses 3 files are in **RawData** format this means binary format with interleaved channels.


In [3]:
#download dataset
localdir, filenames, params = tdc.download_dataset(name='olfactory_bulb')
print(filenames)
print(params)

print()
#create a DataIO
import os, shutil
dirname = 'tridesclous_olfactory_bulb'
if os.path.exists(dirname):
    #remove is already exists
    shutil.rmtree(dirname)    
dataio = DataIO(dirname=dirname)

# feed DataIO
dataio.set_data_source(type='RawData', filenames=filenames, **params)

dataio.add_one_channel_group(channels=list(range(14)))

print(dataio)

    
    

['/home/samuel/Documents/projet/tridesclous/example/olfactory_bulb/OB_file1.raw', '/home/samuel/Documents/projet/tridesclous/example/olfactory_bulb/OB_file2.raw', '/home/samuel/Documents/projet/tridesclous/example/olfactory_bulb/OB_file3.raw']
{'dtype': 'int16', 'sample_rate': 10000.0, 'total_channel': 16, 'bit_to_microVolt': 0.30517578125}

DataIO <id: 140583753418120> 
  workdir: tridesclous_olfactory_bulb
  sample_rate: 10000.0
  total_channel: 16
  channel_groups: 0 [ch0 ch1 ch2 ch3 ... ch10 ch11 ch12 ch13]
  nb_segment: 3
  length: 150000 150000 150000
  durations: 15.0 15.0 15.0 s.


# CatalogueConstructor

In [4]:
cc = CatalogueConstructor(dataio=dataio)
print(cc)

CatalogueConstructor
  chan_grp 0 - ch0 ch1 ch2 ... ch12 ch13
  Signal pre-processing not done yet


## Use automatic parameters and apply the whole chain

tridesclous propose an automatic parameters choice and can apply in one function all the steps.


In [5]:
from pprint import pprint
params = tdc.get_auto_params_for_catalogue(dataio, chan_grp=0)
pprint(params)

{'adjacency_radius_um': 200.0,
 'chunksize': 1000,
 'clean_cluster': {'apply_auto_merge_cluster': True,
                   'apply_auto_split': True,
                   'apply_trash_low_extremum': True,
                   'apply_trash_not_aligned': True,
                   'apply_trash_small_cluster': True},
 'clean_peaks': {'alien_value_threshold': None, 'mode': 'extremum_amplitude'},
 'cluster_kargs': {'adjacency_radius_um': 50.0,
                   'high_adjacency_radius_um': 30.0,
                   'max_loop': 1000,
                   'min_cluster_size': 20},
 'cluster_method': 'pruningshears',
 'duration': 45.0,
 'extract_waveforms': {'wf_left_ms': -1.5, 'wf_right_ms': 2.5},
 'feature_kargs': {'adjacency_radius_um': 50.0, 'n_components_by_channel': 3},
 'feature_method': 'pca_by_channel',
 'make_catalogue': {'inter_sample_oversampling': False,
                    'sparse_thresh_level2': 3,
                    'subsample_ratio': 'auto'},
 'memory_mode': 'memmap',
 'mode': 'sparse',

## apply all catalogue steps

In [6]:
cc.apply_all_steps(params, verbose=True)

estimate_signals_noise 0.15046086600023045
run_signalprocessor 0.7986926489998041
clean_peaks 0.008157486000072822
sample_some_peaks 0.026089860999945813
extract_some_noise 0.061021279000215145
extract_some_features 1.799467378999907
find_clusters 2.0106339809999554
auto_split_cluster 10.9144308609998
trash_not_aligned 0.07785290399988298
auto_merge_cluster 0.33518078700035403
trash_low_extremum 0.015459421999821643
trash_small_cluster 0.015438521999840304
make_catalogue_for_peeler 0.11544246100038436


In [7]:
print(cc)

CatalogueConstructor
  chan_grp 0 - ch0 ch1 ch2 ... ch12 ch13
  nb_peak_by_segment: 4314, 3926, 4564
  some_features.shape: (10720, 42)
  cluster_labels 15 [-11 -1 0 ... 13 14]



## Open CatalogueWindow for visual check

At the end we can save the catalogue.

In [ ]:
%gui qt5
import pyqtgraph as pg
app = pg.mkQApp()
win = tdc.CatalogueWindow(cc)
win.show()
app.exec_()

In [ ]:
# necessary if manual change 
cc.make_catalogue_for_peeler()

# Peeler

Use automatic parameters.


In [8]:
peeler_params = tdc.get_auto_params_for_peelers(dataio, chan_grp=0)
pprint(peeler_params)

{'argmin_method': 'opencl', 'chunksize': 1000, 'engine': 'geometrical'}


In [9]:
catalogue = dataio.load_catalogue()

peeler = Peeler(dataio)
peeler.change_params(catalogue=catalogue, **peeler_params)

t1 = time.perf_counter()
peeler.run()
t2 = time.perf_counter()
print('peeler.run', t2-t1)

print()
for seg_num in range(dataio.nb_segment):
    spikes = dataio.get_spikes(seg_num)
    print('seg_num', seg_num, 'nb_spikes', spikes.size)
    print(spikes[:3])
    


100%|██████████| 150/150 [00:05<00:00, 25.05it/s]

peeler.run 13.46783941800004

seg_num 0 nb_spikes 3982
[( 7, 5, nan) (28, 1, nan) (67, 5, nan)]
seg_num 1 nb_spikes 3471
[( 62, 10, nan) (147,  2, nan) (148, 13, nan)]
seg_num 2 nb_spikes 4498
[(45,  5, nan) (65,  4, nan) (69, 14, nan)]


## Open PeelerWindow for visual checking

In [15]:
%gui qt5
import pyqtgraph as pg
app = pg.mkQApp()
win = tdc.PeelerWindow(dataio=dataio, catalogue=initial_catalogue)
win.show()
app.exec_()


0

Here a snappshot of PeelerWindow

<img src="../doc/img/snapshot_peelerwindow.png">